In [1]:
import cv2
import numpy as np
import xml.etree.ElementTree
from glob import glob
import os
from PIL import Image
import matplotlib.pyplot as plt

In [61]:
def read_xml(xml_path):
    from xml.dom import minidom
    xmldoc = minidom.parse(xml_path)
    itemlist = xmldoc.getElementsByTagName('Coords')
#     print(len(itemlist))
#     print(itemlist[0].attributes['points'].value)
    coord = []
    for s in itemlist:
#         print(s.attributes['points'].value)
        _ = s.attributes['points'].value.split(' ')
        x1 = int(_[0].split(',')[0])
        x2 = int(_[-1].split(',')[0])
        y1 = int(_[0].split(',')[1])
        y2 = int(_[-2].split(',')[1])
        def swap(x1, x2):
            if x1 > x2:
                t_ = x1
                x1 = x2
                x2 = t_
            return x1, x2
        x1, x2 = swap(x1, x2)
        y1, y2 = swap(y1, y2)
        
        coord.append([x1,x2,y1,y2])
    return coord

In [71]:
def extract_table(image_path, xml_path):
    coord = read_xml(xml_path)
    image = cv2.imread(image_path)
    tabels = []
    for x1, x2, y1, y2 in coord:
        tabels.append(image[y1:y2, x1:x2])
    return tabels

In [73]:
len(img_paths)

238

In [75]:
xml_paths = glob('data/*.XML') 
img_paths = glob('data/*.jpg')
k = 0
save_dir = 'tabels'
os.makedirs(save_dir, exist_ok=True)
for image_path, xml_path in zip(img_paths, xml_paths):
    tabels = extract_table(image_path, xml_path)
    name = os.path.split(image_path)[-1]
    for i, tabel in enumerate(tabels):
        cv2.imwrite('tabels/{}_{}'.format(k, name), tabel)
        k += 1
        print('\r {}'.format(k), end='')

 156

In [7]:
paths = glob('tabels/combined_tabel/*.jpg')
names = [os.path.split(path)[-1] for path in paths]
print(names)

['0_eu-0010001.jpg', '10_eu-0030001.jpg', '119_us-0200005.jpg', '142_us-0320001.jpg', '144_us-0330002.jpg', '145_us-0330002.jpg', '147_us-0340002.jpg', '152_us-0370001.jpg', '155_us-0400002.jpg', '3_eu-0010002.jpg', '7_eu-0020001.jpg']


In [53]:
def padding(img):
    w = img.shape[1]
    a = img[:,:w//2]
    b = img[:,w//2:]
    
#     print(a.shape, b.shape)
    h, w = a.shape[:2]
    b = cv2.resize(b, (w, h))
    s_x = np.random.randint(1024 - w)
    s_y = np.random.randint(1024 - h)
    def _(img):
        
        pad = np.ones([1024, 1024, 3], dtype=np.uint8)*255
        pad[s_y:s_y+h, s_x:s_x+w] = img
        return pad
    return np.concatenate([_(a), _(b)], axis=1)

In [65]:
images = [cv2.imread(path) for path in paths]
images_resize = [cv2.resize(img, (0,0), fx=0.5, fy=0.5) for img in images]
images_padding = [padding(img) for img in images_resize]

In [66]:
images_padding = np.array(images_padding)
images_padding = images_padding.reshape([-1, 1024, 2048, 3])

In [70]:
for i, img in enumerate(images_padding):
    cv2.imwrite('{}.png'.format(i),img)